# SOFTKILL-9000: Multi-Agent Motion Capture Simulation

This notebook demonstrates how to run SOFTKILL-9000 in Google Colab.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BkAsDrP/Softkill9000/blob/main/examples/run_in_colab.ipynb)

---

## Overview

SOFTKILL-9000 is a multi-agent reinforcement learning system for motion capture simulation. This notebook will:

1. Install the package from GitHub
2. Run a basic simulation
3. Visualize agent trajectories
4. Demonstrate Q-learning training
5. Show advanced customization options

## 1. Installation

Install SOFTKILL-9000 directly from the GitHub repository:

In [1]:
# Step 1: Install the package from GitHub
!pip install git+https://github.com/BkAsDrP/Softkill9000.git -q
print("✅ SOFTKILL-9000 package installed!")


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
✅ SOFTKILL-9000 package installed!


In [3]:
# Step 2: Upgrade NumPy to resolve dependency conflicts
# Colab comes with NumPy 1.x, but softkill9000 needs NumPy 2.0.x for modern ML libraries
# Pin to <2.1.0 for compatibility with numba, tensorflow, opencv, and cupy
!pip install --upgrade "numpy>=2.0.0,<2.1.0" -q

print("✅ NumPy 2.x installed")
print("")

# Detect if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # Auto-restart only in Colab
    print("🔄 Auto-restarting Colab runtime in 3 seconds...")
    print("   (This prevents binary incompatibility errors)")
    
    import time
    import os
    time.sleep(3)
    os.kill(os.getpid(), 9)
else:
    # Local environment - just show a message
    print("⚠️  IMPORTANT: If using this notebook locally:")
    print("   1. Restart your Jupyter kernel to apply NumPy upgrade")
    print("   2. In VS Code: Click the 'Restart' button in the kernel toolbar")
    print("   3. Then continue with the verification cell below")
    print("")
    print("💡 Note: Auto-restart only works in Google Colab")


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
✅ NumPy 2.x installed

⚠️  IMPORTANT: If using this notebook locally:
   1. Restart your Jupyter kernel to apply NumPy upgrade
   2. In VS Code: Click the 'Restart' button in the kernel toolbar
   3. Then continue with the verification cell below

💡 Note: Auto-restart only works in Google Colab


### ⚠️ Restart Required

**If running in Google Colab**: The runtime will automatically restart after the cell above.

**If running locally** (VS Code, JupyterLab, etc.): Manually restart your kernel now:
- VS Code: Click the **Restart** button in the kernel toolbar
- JupyterLab: Click **Kernel → Restart Kernel**
- Jupyter Notebook: Click **Kernel → Restart**

After restarting, continue with the verification cell below.

In [4]:
# Verification (Run this AFTER runtime restart)
try:
    import softkill9000
    import numpy as np
    
    print(f"✅ SOFTKILL-9000 v{softkill9000.__version__} ready!")
    print(f"✅ NumPy {np.__version__} (compatible with JAX, OpenCV, PyTensor)")
    
    # Test that numpy actually works (catches binary incompatibility)
    test_array = np.random.rand(5)
    np.random.seed(42)
    
    print("")
    print("🎉 Installation complete and verified! Continue with examples below.")
    
except ValueError as e:
    if "numpy.dtype size changed" in str(e):
        print("❌ Binary incompatibility detected!")
        print("")
        print("SOLUTION: Manually restart the runtime:")
        print("   1. Click 'Runtime → Restart runtime' in the menu")
        print("   2. Re-run this cell after restart")
    else:
        raise

✅ SOFTKILL-9000 v1.0.0 ready!
✅ NumPy 2.3.4 (compatible with JAX, OpenCV, PyTensor)

🎉 Installation complete and verified! Continue with examples below.


## 2. Basic Simulation

Let's run a basic simulation with default configuration:

In [ ]:
from softkill9000.simulator import MissionSimulator
from softkill9000.agents.agent import Agent, AgentStats
from softkill9000.environments.environment import CosmicScenario
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Create agents with proper parameters
agents = [
    Agent(
        role="Longsight",
        description="Scout with enhanced mobility and intelligence",
        species="Vyr'khai",
        stats=AgentStats(intelligence=80, mobility=85, tactical=75)
    ),
    Agent(
        role="Lifebinder",
        description="Medic with high empathy and support abilities",
        species="Lumenari",
        stats=AgentStats(empathy=90, intelligence=75, tactical=65)
    ),
    Agent(
        role="Specter",
        description="Assault specialist with superior strength",
        species="Zek'thar",
        stats=AgentStats(strength=95, tactical=80, mobility=70)
    )
]

# Create scenario
scenario = CosmicScenario.generate_random(
    width=100.0,
    height=100.0,
    num_objectives=3,
    num_obstacles=5
)

# Initialize simulator
simulator = MissionSimulator(
    agents=agents,
    scenario=scenario,
    timesteps=20
)

# Run simulation
print("🚀 Starting simulation...\n")
results = simulator.run_simulation()

# Display results
print("\n" + "="*60)
print("📊 SIMULATION RESULTS")
print("="*60)
print(f"Total Reward: {results['total_reward']:.2f}")
print(f"Objectives Completed: {results['objectives_completed']}")
print(f"Average Health: {results['average_health']:.1f}%")
print(f"Final Morale: {results['final_morale']:.1f}%")
print(f"Completion Time: {results['completion_time']:.2f}s")
print("="*60)

TypeError: Agent.__init__() got an unexpected keyword argument 'agent_id'

## 3. Visualize Agent Trajectories

Plot the paths taken by each agent during the mission:

In [ ]:
import matplotlib.pyplot as plt
from softkill9000.visualization import plot_agent_trajectories

# Plot trajectories
fig = plot_agent_trajectories(results['agent_histories'], scenario)
plt.show()

print("\n📍 Agent Movement Summary:")
for agent_id, history in results['agent_histories'].items():
    positions = [(h['position']['x'], h['position']['y']) for h in history]
    start = positions[0]
    end = positions[-1]
    distance = sum(
        np.sqrt((positions[i+1][0] - positions[i][0])**2 + 
                (positions[i+1][1] - positions[i][1])**2)
        for i in range(len(positions)-1)
    )
    print(f"  {agent_id}: {distance:.1f} units traveled")

## 4. Performance Metrics Over Time

Visualize how agent stats evolved during the simulation:

In [ ]:
from softkill9000.visualization import plot_performance_metrics

# Plot performance metrics
fig = plot_performance_metrics(results['agent_histories'])
plt.tight_layout()
plt.show()

print("\n📈 Performance Analysis:")
for agent_id, history in results['agent_histories'].items():
    initial_health = history[0]['health']
    final_health = history[-1]['health']
    health_change = final_health - initial_health
    
    initial_morale = history[0]['morale']
    final_morale = history[-1]['morale']
    morale_change = final_morale - initial_morale
    
    print(f"  {agent_id}:")
    print(f"    Health: {initial_health:.1f} → {final_health:.1f} ({health_change:+.1f})")
    print(f"    Morale: {initial_morale:.1f} → {final_morale:.1f} ({morale_change:+.1f})")

## 5. Q-Learning Training

Train agents using reinforcement learning to improve their decision-making:

In [ ]:
from softkill9000.environments.environment import QLearningTrainer
from softkill9000.config.models import QLearningConfig

# Configure Q-learning
qlearning_config = QLearningConfig(
    learning_rate=0.1,
    discount_factor=0.95,
    exploration_rate=0.2,
    min_exploration_rate=0.01,
    exploration_decay=0.995
)

# Create fresh agents for training
training_agents = [
    Agent(
        role="Longsight",
        description="Scout trainee",
        species="Vyr'khai",
        stats=AgentStats(intelligence=75, mobility=80)
    ),
    Agent(
        role="Lifebinder",
        description="Medic trainee",
        species="Lumenari",
        stats=AgentStats(empathy=85, intelligence=70)
    ),
    Agent(
        role="Specter",
        description="Assault trainee",
        species="Zek'thar",
        stats=AgentStats(strength=90, tactical=75)
    )
]

# Initialize trainer
trainer = QLearningTrainer(config=qlearning_config)

# Training loop
print("🎓 Training agents with Q-Learning...\n")
episodes = 50
rewards_history = []

for episode in range(episodes):
    # Create new scenario for each episode
    training_scenario = CosmicScenario.generate_random(
        width=100.0,
        height=100.0,
        num_objectives=3,
        num_obstacles=5
    )
    
    # Run training episode
    sim = MissionSimulator(
        agents=training_agents,
        scenario=training_scenario,
        timesteps=20
    )
    
    results = sim.run_simulation()
    episode_reward = results['total_reward']
    rewards_history.append(episode_reward)
    
    # Train agents based on results
    for agent in training_agents:
        # Simplified training (in practice, you'd pass state-action-reward tuples)
        state = hash((agent.position[0], agent.position[1], agent.stats.strength))
        action = 'advance'  # Simplified action
        next_state = hash((agent.position[0] + 1, agent.position[1], agent.stats.strength))
        
        trainer.train_agent(
            agent=agent,
            state=state,
            action=action,
            reward=episode_reward / len(training_agents),
            next_state=next_state
        )
    
    if (episode + 1) % 10 == 0:
        avg_reward = np.mean(rewards_history[-10:])
        print(f"Episode {episode + 1}/{episodes} - Avg Reward (last 10): {avg_reward:.2f}")

# Plot training progress
plt.figure(figsize=(10, 6))
plt.plot(rewards_history, alpha=0.6, label='Episode Reward')
plt.plot(np.convolve(rewards_history, np.ones(10)/10, mode='valid'), 
         linewidth=2, label='Moving Average (10)')
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.title('Q-Learning Training Progress')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"\n✅ Training completed!")
print(f"Final exploration rate: {trainer.config.exploration_rate:.3f}")
print(f"Average reward (last 10 episodes): {np.mean(rewards_history[-10:]):.2f}")

## 6. Custom Configuration

Use YAML configuration for advanced customization:

In [ ]:
import yaml

# For this example, we'll create agents manually since the config structure
# doesn't directly map to the Agent dataclass
print("⚙️  Creating custom agent configurations...\n")

# Define custom agents with different stats
custom_agents = [
    Agent(
        role="Longsight",
        description="Alpha scout with enhanced mobility",
        species="Vyr'khai",
        stats=AgentStats(intelligence=85, mobility=90, tactical=80, strength=55, empathy=60)
    ),
    Agent(
        role="Specter",
        description="Bravo assault with superior firepower",
        species="Zek'thar",
        stats=AgentStats(strength=100, tactical=85, mobility=75, intelligence=65, empathy=50)
    ),
    Agent(
        role="Lifebinder",
        description="Charlie medic with maximum support",
        species="Lumenari",
        stats=AgentStats(empathy=95, intelligence=80, tactical=70, strength=45, mobility=65)
    )
]

print(f"✅ Created {len(custom_agents)} custom agents:")
for agent in custom_agents:
    print(f"   - {agent.role} ({agent.species}): {agent.description}")

# Create custom scenario (larger and more challenging)
custom_scenario = CosmicScenario.generate_random(
    width=150.0,
    height=150.0,
    num_objectives=5,
    num_obstacles=8
)

print(f"\n🗺️  Custom Scenario:")
print(f"   Size: 150x150 units")
print(f"   Objectives: 5")
print(f"   Obstacles: 8")

# Run custom simulation with more timesteps
custom_sim = MissionSimulator(
    agents=custom_agents,
    scenario=custom_scenario,
    timesteps=30  # Longer mission
)

print("\n🚀 Running custom simulation (30 timesteps)...")
custom_results = custom_sim.run_simulation()

print("\n📊 Custom Simulation Results:")
print(f"  Total Reward: {custom_results['total_reward']:.2f}")
print(f"  Objectives Completed: {custom_results['objectives_completed']}/5")
print(f"  Average Health: {custom_results['average_health']:.1f}%")
print(f"  Final Morale: {custom_results['final_morale']:.1f}%")
print(f"  Completion Time: {custom_results['completion_time']:.2f}s")

## 7. Compare Multiple Scenarios

Run multiple simulations and compare results:

In [ ]:
# Run multiple scenarios
print("🔄 Running multiple scenarios...\n")

scenarios_data = []
num_runs = 10

role_config = [
    ("Longsight", "Scout", "Vyr'khai", AgentStats(intelligence=75, mobility=80)),
    ("Lifebinder", "Medic", "Lumenari", AgentStats(empathy=85, intelligence=70)),
    ("Specter", "Assault", "Zek'thar", AgentStats(strength=90, tactical=75))
]

for run in range(num_runs):
    # Create new agents and scenario for each run
    agents = [
        Agent(
            role=role_config[i % 3][0],
            description=f"{role_config[i % 3][1]} agent #{run}-{i}",
            species=role_config[i % 3][2],
            stats=role_config[i % 3][3]
        )
        for i in range(3)
    ]
    
    scenario = CosmicScenario.generate_random(
        width=100.0,
        height=100.0,
        num_objectives=3,
        num_obstacles=5
    )
    
    sim = MissionSimulator(agents=agents, scenario=scenario, timesteps=20)
    results = sim.run_simulation()
    
    scenarios_data.append({
        'run': run + 1,
        'reward': results['total_reward'],
        'objectives': results['objectives_completed'],
        'health': results['average_health'],
        'time': results['completion_time']
    })

# Analyze results
import pandas as pd

df = pd.DataFrame(scenarios_data)

print("📊 Statistical Summary:")
print(df.describe())

# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].bar(df['run'], df['reward'], color='steelblue')
axes[0, 0].axhline(df['reward'].mean(), color='red', linestyle='--', label='Mean')
axes[0, 0].set_xlabel('Run')
axes[0, 0].set_ylabel('Total Reward')
axes[0, 0].set_title('Reward by Scenario')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].bar(df['run'], df['objectives'], color='green')
axes[0, 1].axhline(df['objectives'].mean(), color='red', linestyle='--', label='Mean')
axes[0, 1].set_xlabel('Run')
axes[0, 1].set_ylabel('Objectives Completed')
axes[0, 1].set_title('Objectives by Scenario')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].bar(df['run'], df['health'], color='orange')
axes[1, 0].axhline(df['health'].mean(), color='red', linestyle='--', label='Mean')
axes[1, 0].set_xlabel('Run')
axes[1, 0].set_ylabel('Average Health (%)')
axes[1, 0].set_title('Final Health by Scenario')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].bar(df['run'], df['time'], color='purple')
axes[1, 1].axhline(df['time'].mean(), color='red', linestyle='--', label='Mean')
axes[1, 1].set_xlabel('Run')
axes[1, 1].set_ylabel('Time (seconds)')
axes[1, 1].set_title('Completion Time by Scenario')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✅ Completed {num_runs} scenario runs")
print(f"Average reward: {df['reward'].mean():.2f} ± {df['reward'].std():.2f}")

## 🎯 Next Steps

Explore more features:

1. **REST API**: Deploy the FastAPI server for remote access
2. **Custom Agents**: Create agents with specialized behaviors
3. **Advanced Scenarios**: Design complex mission environments
4. **Extended Training**: Train agents for more episodes to see performance improvements
5. **Visualization**: Create custom plots and animations

### Documentation

- [Architecture Guide](https://github.com/BkAsDrP/Softkill9000/blob/main/docs/architecture.md)
- [API Reference](https://github.com/BkAsDrP/Softkill9000/blob/main/docs/api_reference.md)
- [User Guide](https://github.com/BkAsDrP/Softkill9000/blob/main/docs/user_guide.md)
- [Deployment Guide](https://github.com/BkAsDrP/Softkill9000/blob/main/docs/deployment.md)

### Repository

- GitHub: [BkAsDrP/Softkill9000](https://github.com/BkAsDrP/Softkill9000)
- Issues: [Report bugs or request features](https://github.com/BkAsDrP/Softkill9000/issues)
- License: MIT

---

**Happy simulating! 🚀**